<a href="https://colab.research.google.com/github/maithoi/thesis/blob/main/convert_label_to_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries & dependences 


In [ ]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

# Connect with Google Drive 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 0.Thesis
 1.Career
 22feb-20mar.gsheet
'9-DCMH - Mang may tinh - 405015.gdoc'
'Beat Task1.gdoc'
'Beautiful words.gsheet'
'Big Pictures.gjam'
 BME-Assignment.gdoc
'Cal Newport - So good they can'\''t ignore you_ why skills trump passion in the quest for work you love-Business Plus (2012).mobi'
 Careeer.gdoc
 CAREERS
'Chapter10 - Email.gslides'
'Chapter11-Network Managment-3ed.gslides'
'Chapter1 - Data link Layer (1).gslides'
'Chapter1 - Data link Layer.gslides'
 Chapter1-Introduction.gslides
'Chapter2- Mac Protocol.gslides'
'Chapter3 - Spanning Tree.gslides'
'Chapter4 - Internet Protocol.gslides'
'Chapter5 - Routing.gslides'
'Chapter6 - Transport Layer.gslides'
'Chapter7 - DNS.gslides'
'Chapter8 - FTP (1).gslides'
'Chapter8 - FTP (2).gslides'
'Chapter8 - FTP (3).gslides'
'Chapter8 - FTP (4).gslides'
'Chapter8 - FTP.gslides'
'Chapter9 - HTTP.gslides'
'Chrome OS Cloud backup'
'Coding Interview 

# Download Yolov5 Model 

In [ ]:
!git clone https://github.com/rkuo2000/yolov5
%cd yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


In [ ]:
!mkdir -p Dataset/FaceMask/Images
!mkdir -p Dataset/FaceMask/Labels

In [ ]:
!pwd

/content/yolov5


# Copy image files 

In [ ]:
# upload files into Dataset/FaceMask/Images 
from google.colab import files
uploaded = files.upload()

In [ ]:
!cp -rf /content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label Dataset/FaceMask/Labels

# Create destinations images path & labels path 

In [ ]:
!mkdir -p Dataset/images Dataset/labels

# Create Dataset 

In [ ]:
FILE_ROOT = "/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/"
IMAGE_PATH = FILE_ROOT + "500image"  
ANNOTATIONS_PATH = FILE_ROOT + "500label"

DATA_ROOT = "Dataset/"
LABELS_ROOT = DATA_ROOT + "FaceMask/Labels"
IMAGES_ROOT = DATA_ROOT + "FaceMask/Images"  

DEST_IMAGES_PATH = "images"
DEST_LABELS_PATH = "labels" 

In [ ]:
classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

### convert annotations (from COCO .xml to YOLO format .txt)

In [ ]:
def cord_converter(size, box):
    """
    convert xml annotation to darknet format coordinates
    :param size： [w,h]
    :param box: anchor box coordinates [upper-left x,uppler-left y,lower-right x, lower-right y]
    :return: converted [x,y,w,h]
    """
    x1 = int(box[0])
    y1 = int(box[1])
    x2 = int(box[2])
    y2 = int(box[3])

    dw = np.float32(1. / int(size[0]))
    dh = np.float32(1. / int(size[1]))

    w = x2 - x1
    h = y2 - y1
    x = x1 + (w / 2)
    y = y1 + (h / 2)

    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def save_file(img_jpg_file_name, size, img_box):
    save_file_name = LABELS_ROOT + '/' + img_jpg_file_name + '.txt'
    print(save_file_name)
    file_path = open(save_file_name, "a+")
    for box in img_box:

        cls_num = classes.index(box[0])

        new_box = cord_converter(size, box[1:])

        file_path.write(f"{cls_num} {new_box[0]} {new_box[1]} {new_box[2]} {new_box[3]}\n")

    file_path.flush()
    file_path.close()
    
def get_xml_data(file_path, img_xml_file):
    img_path = file_path + '/' + img_xml_file + '.xml'
    print(img_path)

    dom = parse(img_path)
    root = dom.documentElement
    img_name = root.getElementsByTagName("filename")[0].childNodes[0].data
    img_size = root.getElementsByTagName("size")[0]
    objects = root.getElementsByTagName("object")
    img_w = img_size.getElementsByTagName("width")[0].childNodes[0].data
    img_h = img_size.getElementsByTagName("height")[0].childNodes[0].data
    img_c = img_size.getElementsByTagName("depth")[0].childNodes[0].data
    # print("img_name:", img_name)
    # print("image_info:(w,h,c)", img_w, img_h, img_c)
    img_box = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        x1 = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        y1 = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        x2 = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        y2 = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)
        # print("box:(c,xmin,ymin,xmax,ymax)", cls_name, x1, y1, x2, y2)
        img_jpg_file_name = img_xml_file + '.jpg'
        img_box.append([cls_name, x1, y1, x2, y2])
    # print(img_box)

    # test_dataset_box_feature(img_jpg_file_name, img_box)
    save_file(img_xml_file, [img_w, img_h], img_box)

In [ ]:
print(os.listdir("/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label"))

['IMG_019755.xml', 'IMG_019750.xml', 'IMG_019680.xml', 'IMG_019745.xml', 'IMG_019690.xml', 'IMG_019670.xml', 'IMG_019660.xml', 'IMG_019650.xml', 'IMG_019725.xml', 'IMG_019740.xml', 'IMG_019710.xml', 'IMG_019665.xml', 'IMG_019675.xml', 'IMG_019715.xml', 'IMG_019685.xml', 'IMG_019735.xml', 'IMG_019655.xml', 'IMG_019730.xml', 'IMG_019695.xml', 'IMG_0104150.xml', 'IMG_0104115.xml', 'IMG_0104110.xml', 'IMG_0103145.xml', 'IMG_0103115.xml', 'IMG_0103120.xml', 'IMG_0103160.xml', 'IMG_0103155.xml', 'IMG_0103110.xml', 'IMG_0103125.xml', 'IMG_0103165.xml', 'IMG_037185.xml', 'IMG_0103130.xml', 'IMG_0103105.xml', 'IMG_0103135.xml', 'IMG_0103100.xml', 'IMG_0103170.xml', 'IMG_0104105.xml', 'IMG_0103140.xml', 'IMG_0104100.xml', 'IMG_0103150.xml', 'IMG_036655.xml', 'IMG_031335.xml', 'IMG_019985.xml', 'IMG_031345.xml', 'IMG_031375.xml', 'IMG_031350.xml', 'IMG_031380.xml', 'IMG_031370.xml', 'IMG_031325.xml', 'IMG_031360.xml', 'IMG_031395.xml', 'IMG_031330.xml', 'IMG_019995.xml', 'IMG_031390.xml', 'IMG_03

In [ ]:
files = os.listdir(ANNOTATIONS_PATH)
for file in files:
    print("file name: ", file)
    file_xml = file.split(".")
    get_xml_data(ANNOTATIONS_PATH, file_xml[0])

file name:  IMG_019755.xml
/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label/IMG_019755.xml
Dataset/FaceMask/Labels/IMG_019755.txt
file name:  IMG_019750.xml
/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label/IMG_019750.xml
Dataset/FaceMask/Labels/IMG_019750.txt
file name:  IMG_019680.xml
/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label/IMG_019680.xml
Dataset/FaceMask/Labels/IMG_019680.txt
file name:  IMG_019745.xml
/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label/IMG_019745.xml
Dataset/FaceMask/Labels/IMG_019745.txt
file name:  IMG_019690.xml
/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label/IMG_019690.xml
Dataset/FaceMask/Labels/IMG_019690.txt
file name:  IMG_019670.xml
/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label/IMG_019670.xml
Dataset/FaceMask/Labels/IMG_019670.txt
file name:  IMG_019660.xml
/content/gdrive/MyDrive/0.Thesis/data/face_mask_detection/500label/IMG_019660.xml
Dat

# Download converted YOLO TXT

In [ ]:
# !zip -r /content/file.zip /content/Folder_To_Zip
!zip -r /content/labels.zip /content/yolov5/Dataset/FaceMask/Labels

  adding: content/yolov5/Dataset/FaceMask/Labels/ (stored 0%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_019875.txt (deflated 28%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_019755.txt (deflated 28%)
  adding: content/yolov5/Dataset/FaceMask/Labels/c100.txt (deflated 28%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_019625.txt (deflated 42%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_0198295.txt (deflated 27%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_0199110.txt (deflated 28%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_019630.txt (deflated 42%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_010395.txt (deflated 28%)
  adding: content/yolov5/Dataset/FaceMask/Labels/a50.txt (deflated 47%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_0197145.txt (deflated 28%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_0196110.txt (deflated 27%)
  adding: content/yolov5/Dataset/FaceMask/Labels/IMG_0199170.txt (deflated 41%

# Split images dataset 

In [ ]:
from sklearn.model_selection import train_test_split
image_list = os.listdir('Dataset/FaceMask/Images')
train_list, test_list = train_test_split(image_list, test_size=0.2, random_state=7)
val_list, test_list = train_test_split(test_list, test_size=0.5, random_state=8)

print('total =',len(image_list))
print('train :',len(train_list))
print('val   :',len(val_list))
print('test  :',len(test_list))